In [1]:
# !pip install fastapi
# !pip install uvicorn
# !pip install pydantic
# !pip install scikit-learn
# !pip install requests
# !pip install pypi-json
# !pip install pyngrok
# !pip install nest-asyncio

In [2]:
!ngrok config add-authtoken 32L4iiG20IL14J4VSRrwSPHaM0v_6xEiWgMmfpmHzeHmaAX9e

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
import pickle
import json
import uvicorn
import threading
import time
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok

In [4]:
app = FastAPI()
origins = ['*']
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

In [5]:
class model_input(BaseModel):
    Pregnancies: int
    Glucose: int
    BloodPressure: int
    SkinThickness: int
    Insulin: int
    BMI: float
    DiabetesPedigreeFunction: float
    Age: int

In [6]:
diabetes_model = pickle.load(open('retrained_model.sav', 'rb'))

@app.post('/diabetes_prediction')
def diabetes_pred(input_parameters: model_input):
    input_dictionary = input_parameters.dict()

    preg = input_dictionary['Pregnancies']
    glu = input_dictionary['Glucose']
    bp = input_dictionary['BloodPressure']
    skin = input_dictionary['SkinThickness']
    insulin = input_dictionary['Insulin']
    bmi = input_dictionary['BMI']
    dpf = input_dictionary['DiabetesPedigreeFunction']
    age = input_dictionary['Age']

    input_list = [preg, glu, bp, skin, insulin, bmi, dpf, age]

    prediction = diabetes_model.predict([input_list])

    if prediction[0] == 0:
        return {'prediction': 'The person is not diabetic'}
    else:
        return {'prediction': 'The person is diabetic'}

In [7]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

def run_server():
    uvicorn.run(app, port=8000, host='0.0.0.0')

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Shutting down server.")
    ngrok.kill()

INFO:     Started server process [6971]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://e419e9adc6c6.ngrok-free.app
Shutting down server.
